## 0 Global Settings and Import

In [ ]:
%pip install --upgrade blinker
%pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.3/355.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.0/115.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.0/537.0 kB 34.1 MB/s 

In [ ]:
import sys
import recommenders
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import logging
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

from recommenders.models.rbm.rbm import RBM
from recommenders.datasets.python_splitters import numpy_stratified_split
from recommenders.datasets.sparse import AffinityMatrix
from recommenders.utils.timer import Timer
from recommenders.utils.plot import line_graph
from recommenders.datasets import movielens
from recommenders.evaluation.python_evaluation import (
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    rmse,
    mae,
)

#For interactive mode only
# %load_ext autoreload
# %autoreload 2

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Pandas version: 2.2.2
Tensorflow version: 2.19.0


/usr/local/lib/python3.12/dist-packages/pandera/_pandas_deprecated.py:160: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


In [ ]:
!pip install  surprise


In [ ]:
# Downgrade numpy to a compatible version
!pip install "numpy<2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cornac 2.3.5 requires numpy>2.0.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; pyt

# Data preparation and inspection


Below, we first load the  data in pandas dataframes, explain how the user/affinity matrix is built and how the train/test set is generated.

In [ ]:
df=pd.read_csv("/content/collected data.csv")
df.rename(columns = {'user_id':'userID', 'restaurant_id':'itemID'}, inplace = True)


# Encode user and item IDs
from sklearn import preprocessing

user_encoder = preprocessing.LabelEncoder()
df['userID'] = user_encoder.fit_transform(df['userID'])

item_encoder = preprocessing.LabelEncoder()

df['itemID'] = item_encoder.fit_transform(df['itemID'])
header = {
        "col_user": "userID",
        "col_item": "itemID",
        "col_rating": "rating",
    }
df.head()
df.describe()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,userID,User Longitude,User Latitude,Resturant Longitude,Resturant Latitude,rating,itemID
count,6476.000000,6476.000000,6476.000000,6476.000000,6476.000000,6476.000000,6476.000000
mean,695.389284,46.144012,23.463113,45.331269,24.241446,3.755420,15.935300
std,259.155075,5.484979,2.108800,4.580936,1.998283,0.610533,9.549836
min,0.000000,39.100104,19.211657,39.109269,21.527526,1.100000,0.000000
25%,579.000000,39.283388,21.484644,39.153422,21.604568,3.500000,8.000000
50%,735.500000,46.824416,24.191063,46.681993,24.718261,3.800000,16.000000
75%,894.000000,49.953170,24.781044,50.118295,26.291847,4.100000,24.000000
max,1061.000000,55.598743,29.059250,50.220734,27.129412,5.000000,33.000000


## Split the data using the stratified splitter  



In [ ]:
# Instantiate the splitter with the preprocessed  DataFrame
am1m = AffinityMatrix(df=df[['userID', 'itemID', 'rating']], **header)

# Obtain the full sparse matrix for the full dataset
X1m, _, _ = am1m.gen_affinity_matrix()

Next, we split the matrix above into train and test set sparse matrices

In [ ]:
Xtr_1m, Xtst_1m = numpy_stratified_split(X1m)

## Model application, performance and analysis of the results  



In [ ]:
#First we initialize the model class
model_1m = RBM(
    possible_ratings=np.setdiff1d(np.unique(Xtr_1m), np.array([0])),
    visible_units=Xtr_1m.shape[1],
    hidden_units=1200,
    training_epoch=30,
    minibatch_size=350,
    with_metrics=True
)

In [ ]:
#Model Fit
with Timer() as train_time:
    model_1m.fit(Xtr_1m)

print("Took {:.2f} seconds for training.".format(train_time.interval))



## Model Evaluation

To evaluate the model performance and compare it against the other algorithms in this repository, we use the `recommend_k_items()` method. Note that we pass 'maps' as a second argument in order to return the correct user/item IDs in a pandas dataframe format.  

In [ ]:
#collection of evaluation metrics
def ranking_metrics(
    data_size,
    data_true,
    data_pred,
    K
):

    eval_map = map_at_k(data_true, data_pred, col_user="userID", col_item="itemID",
                    col_rating="rating", col_prediction="prediction",
                    relevancy_method="top_k", k= K)

    eval_ndcg = ndcg_at_k(data_true, data_pred, col_user="userID", col_item="itemID",
                      col_rating="rating", col_prediction="prediction",
                      relevancy_method="top_k", k= K)

    eval_precision = precision_at_k(data_true, data_pred, col_user="userID", col_item="itemID",
                               col_rating="rating", col_prediction="prediction",
                               relevancy_method="top_k", k= K)

    eval_recall = recall_at_k(data_true, data_pred, col_user="userID", col_item="itemID",
                          col_rating="rating", col_prediction="prediction",
                          relevancy_method="top_k", k= K)

    # Filter predictions to only include items present in the true ratings for RMSE and MAE calculation
    merged_df = pd.merge(data_true[['userID', 'itemID', 'rating']], data_pred[['userID', 'itemID', 'prediction']], on=['userID', 'itemID'], how='inner')

    # Drop rows with NaN values in rating or prediction columns
    merged_df.dropna(subset=['rating', 'prediction'], inplace=True)

    eval_rmse = rmse(merged_df, merged_df, col_user="userID", col_item="itemID",
                          col_rating="rating", col_prediction="prediction")
    eval_mae = mae(merged_df, merged_df, col_user="userID", col_item="itemID",
                          col_rating="rating", col_prediction="prediction")


    df_result = pd.DataFrame(
        {   "Dataset": data_size,
            "K": K,
            "MAP": eval_map,
            "nDCG@k": eval_ndcg,
            "Precision@k": eval_precision,
            "Recall@k": eval_recall,
            "RMSE": eval_rmse,
            "MAE": eval_mae,
        },
        index=[0]
    )

    return df_result

In [ ]:

#Model prediction on the test set Xtst.
with Timer() as prediction_time:
    top_k_1m =  model_1m.recommend_k_items(Xtst_1m)

print("Took {:.2f} seconds for prediction.".format(prediction_time.interval))

Took 2.90 seconds for prediction.


In [ ]:
top_k_df_1m = am1m.map_back_sparse(top_k_1m, kind = 'prediction')
test_df_1m = am1m.map_back_sparse(Xtst_1m, kind = 'ratings')

In [ ]:

test_df_1m = test_df_1m.rename(columns={"user_id": "userID", "restaurant_id": "itemID"})
top_k_df_1m = top_k_df_1m.rename(columns={"user_id": "userID", "restaurant_id": "itemID"})

rating_1m= ranking_metrics(
    data_true =test_df_1m,
    data_pred =top_k_df_1m,
    K =10)

rating_1m



,Dataset,K,MAP,nDCG@k,Precision@k,Recall@k,RMSE,MAE
0,opti,10,0.976778,0.98276,0.199748,0.996222,0.889172,0.780267


In [ ]:

rating_1m= ranking_metrics(
    data_true =test_df_1m,
    data_pred =top_k_df_1m,
    K =10)

rating_1m


,Dataset,K,MAP,nDCG@k,Precision@k,Recall@k,RMSE,MAE
0,mv 1m,10,0.978073,0.983439,0.199748,0.996222,0.882915,0.779332


In [ ]:

rating_1m= ranking_metrics(
    data_true =test_df_1m,
    data_pred =top_k_df_1m,
    K =15)

rating_1m

,Dataset,K,MAP,nDCG@k,Precision@k,Recall@k,RMSE,MAE
0,mv 1m,15,0.979349,0.984176,0.132997,0.994332,0.890592,0.782611


## Repeat the experiment with cross validation  



In [ ]:
import numpy as np
import pandas as pd
from surprise.model_selection import KFold
from surprise import Dataset, Reader
from recommenders.models.rbm.rbm import RBM
from recommenders.datasets.sparse import AffinityMatrix
from recommenders.utils.timer import Timer
from recommenders.evaluation.python_evaluation import (
    map_at_k, ndcg_at_k, precision_at_k, recall_at_k, rmse, mae, rsquared, exp_var
)
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions
# This ensures the AffinityMatrix operates on a pandas DataFrame and gets user/item mappings
am1m = AffinityMatrix(df=df[['userID', 'itemID', 'rating']], **header)

# Obtain the full sparse matrix (X_full_rbm) and mappings for consistent sizing
# This is done once to get the overall dimensions and mappings
X_full_rbm, user_map_rbm, item_map_rbm = am1m.gen_affinity_matrix()
num_total_users = X_full_rbm.shape[0]
num_total_items = X_full_rbm.shape[1]

# Prepare data for Surprise library's KFold
# Surprise needs a DataFrame with user, item, rating columns
surprise_df = df[['userID', 'itemID', 'rating']].copy()
reader = Reader(rating_scale=(1, 5))  # Adjust based on your rating scale
surprise_data = Dataset.load_from_df(surprise_df, reader)

# Define a cross-validation iterator (n_splits reduced for faster execution during debugging)
kf = KFold(n_splits=10, random_state=42) # Using random_state for reproducibility

# Lists to store metrics for each fold
rmse_scores = []
mae_scores = []
rsquared_scores = []
exp_var_scores = []
map_scores = []
ndcg_scores = []
precision_scores = []
recall_scores = []

# We will collect all predictions to ensure consistent evaluation across folds
all_test_true_ratings = []
all_test_predicted_ratings = []

for fold_idx, (trainset, testset) in enumerate(kf.split(surprise_data)):
    print(f"--- Fold {fold_idx + 1} ---")

    # Convert Surprise trainset to RBM-compatible sparse matrix (NumPy array)
    Xtr_rbm = np.zeros((num_total_users, num_total_items))
    for inner_uid, inner_iid, rating in trainset.all_ratings():
        raw_uid = trainset.to_raw_uid(inner_uid)
        raw_iid = trainset.to_raw_iid(inner_iid)
        # Use the returned user_map_rbm and item_map_rbm directly
        user_idx = user_map_rbm[raw_uid]
        item_idx = item_map_rbm[raw_iid]
        Xtr_rbm[user_idx, item_idx] = rating

    # Convert Surprise testset to RBM-compatible sparse matrix (NumPy array)
    Xtst_rbm = np.zeros((num_total_users, num_total_items))
    test_ratings_list_fold = [] # Collect true ratings for this fold
    # Iterate over testset to populate Xtst_rbm and test_ratings_list_fold
    # The testset yields raw_uid, raw_iid, and the true rating
    for raw_uid, raw_iid, true_r in testset:
        # Ensure the raw_uid and raw_iid exist in the overall mapping
        if raw_uid in user_map_rbm and raw_iid in item_map_rbm: # Check against the direct maps
            user_idx = user_map_rbm[raw_uid] # Use the direct map
            item_idx = item_map_rbm[raw_iid] # Use the direct map
            Xtst_rbm[user_idx, item_idx] = true_r
            test_ratings_list_fold.append({'userID': raw_uid, 'itemID': raw_iid, 'rating': true_r})
    test_df_for_eval = pd.DataFrame(test_ratings_list_fold)

    # Initialize the RBM model for the current fold
    model_1m = RBM(
        possible_ratings=np.setdiff1d(np.unique(Xtr_rbm[Xtr_rbm != 0]), np.array([0])),
        visible_units=num_total_items,
        hidden_units=1200,
        training_epoch=30,
        minibatch_size=350,
        with_metrics=True
    )

    # Train the model
    with Timer() as train_time:
        model_1m.fit(Xtr_rbm)
    print("Took {:.2f} seconds for training.".format(train_time.interval))

    # Model prediction on the test set
    with Timer() as prediction_time:
        top_k_1m = model_1m.recommend_k_items(Xtst_rbm)
    print("Took {:.2f} seconds for prediction.".format(prediction_time.interval))

    # Convert recommended items to DataFrame for evaluation
    top_k_df_1m = am1m.map_back_sparse(top_k_1m, kind='prediction')

    # Filter top_k_df_1m to only include items that were actually in the test_df_for_eval for direct comparison
    # This is crucial for RMSE/MAE calculations to avoid comparing predictions for items not in the test set
    merged_predictions = pd.merge(
        test_df_for_eval[['userID', 'itemID', 'rating']],
        top_k_df_1m[['userID', 'itemID', 'prediction']],
        on=['userID', 'itemID'],
        how='inner'
    )

    # Ensure prediction column has numeric type, fill NaN if any (though inner merge should prevent most)
    merged_predictions['prediction'] = pd.to_numeric(merged_predictions['prediction'], errors='coerce').fillna(0)

    # Evaluate using recommenders evaluation functions
    eval_rmse = rmse(test_df_for_eval, merged_predictions, col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction')
    eval_mae = mae(test_df_for_eval, merged_predictions, col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction')
    eval_rsquared = rsquared(test_df_for_eval, merged_predictions, col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction')
    eval_exp_var = exp_var(test_df_for_eval, merged_predictions, col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction')

    k = 10
    eval_map = map_at_k(test_df_for_eval, top_k_df_1m, col_prediction='prediction', k=k)
    eval_ndcg = ndcg_at_k(test_df_for_eval, top_k_df_1m, col_prediction='prediction', k=k)
    eval_precision = precision_at_k(test_df_for_eval, top_k_df_1m, col_prediction='prediction', k=k)
    eval_recall = recall_at_k(test_df_for_eval, top_k_df_1m, col_prediction='prediction', k=k)

    rmse_scores.append(eval_rmse)
    mae_scores.append(eval_mae)
    rsquared_scores.append(eval_rsquared)
    exp_var_scores.append(eval_exp_var)
    map_scores.append(eval_map)
    ndcg_scores.append(eval_ndcg)
    precision_scores.append(eval_precision)
    recall_scores.append(eval_recall)

    print("Microsoft Recommenders Evaluation:")
    print("RMSE:\t\t%f" % eval_rmse,
          "MAE:\t\t%f" % eval_mae,
          "rsquared:\t%f" % eval_rsquared,
          "exp var:\t%f" % eval_exp_var, sep='\n')

    print('----')

    print("MAP:\t%f" % eval_map,
          "NDCG:\t%f" % eval_ndcg,
          "Precision@K:\t%f" % eval_precision,
          "Recall@K:\t%f" % eval_recall, sep='\n')
    print("\n" + "="*30 + "\n") # Separator for readability

print("\n=== Average Metrics Across Folds ===")

metrics_data = {
    "Metric": [
        "RMSE",
        "MAE",
        "R-squared",
        "Explained Variance",
        "MAP@10",
        "NDCG@10",
        "Precision@10",
        "Recall@10"
    ],
    "Value": [
        f"{np.mean(rmse_scores):.4f} \u00B1 {np.std(rmse_scores):.4f}",
        f"{np.mean(mae_scores):.4f} \u00B1 {np.std(mae_scores):.4f}",
        f"{np.mean(rsquared_scores):.4f} \u00B1 {np.std(rsquared_scores):.4f}",
        f"{np.mean(exp_var_scores):.4f} \u00B1 {np.std(exp_var_scores):.4f}",
        f"{np.mean(map_scores):.4f} \u00B1 {np.std(map_scores):.4f}",
        f"{np.mean(ndcg_scores):.4f} \u00B1 {np.std(ndcg_scores):.4f}",
        f"{np.mean(precision_scores):.4f} \u00B1 {np.std(precision_scores):.4f}",
        f"{np.mean(recall_scores):.4f} \u00B1 {np.std(recall_scores):.4f}"
    ]
}

df_metrics = pd.DataFrame(metrics_data)
display(df_metrics)

print("\n=== Average Metrics (Mean \u00B1 Std) ===")
print(f"RMSE: {np.mean(rmse_scores):.4f} \u00B1 {np.std(rmse_scores):.4f}")
print(f"MAE: {np.mean(mae_scores):.4f} \u00B1 {np.std(mae_scores):.4f}")
print(f"R-squared: {np.mean(rsquared_scores):.4f} \u00B1 {np.std(rsquared_scores):.4f}")
print(f"Explained Variance: {np.mean(exp_var_scores):.4f} \u00B1 {np.std(exp_var_scores):.4f}")
print(f"MAP@10: {np.mean(map_scores):.4f} \u00B1 {np.std(map_scores):.4f}")
print(f"NDCG@10: {np.mean(ndcg_scores):.4f} \u00B1 {np.std(ndcg_scores):.4f}")
print(f"Precision@10: {np.mean(precision_scores):.4f} \u00B1 {np.std(precision_scores):.4f}")
print(f"Recall@10: {np.mean(recall_scores):.4f} \u00B1 {np.std(recall_scores):.4f}")

--- Fold 1 ---
Took 7.14 seconds for training.
Took 0.15 seconds for prediction.
Microsoft Recommenders Evaluation:
RMSE:		0.814230
MAE:		0.710107
rsquared:	-0.967986
exp var:	0.333414
----
MAP:	0.988068
NDCG:	0.991411
Precision@K:	0.147273
Recall@K:	1.000000


--- Fold 2 ---
Took 3.56 seconds for training.
Took 0.16 seconds for prediction.
Microsoft Recommenders Evaluation:
RMSE:		0.810852
MAE:		0.707976
rsquared:	-1.459686
exp var:	0.253985
----
MAP:	0.990280
NDCG:	0.992909
Precision@K:	0.146606
Recall@K:	1.000000


--- Fold 3 ---
Took 5.03 seconds for training.
Took 0.16 seconds for prediction.
Microsoft Recommenders Evaluation:
RMSE:		0.800473
MAE:		0.695653
rsquared:	-0.827837
exp var:	0.326546
----
MAP:	0.985893
NDCG:	0.989419
Precision@K:	0.144843
Recall@K:	0.996637


--- Fold 4 ---
Took 3.59 seconds for training.
Took 0.16 seconds for prediction.
Microsoft Recommenders Evaluation:
RMSE:		0.806622
MAE:		0.693544
rsquared:	-0.733966
exp var:	0.348529
----
MAP:	0.980624
NDCG:	0.98

,Metric,Value
0,RMSE,0.8151 ± 0.0087
1,MAE,0.7095 ± 0.0113
2,R-squared,-0.8907 ± 0.2790
3,Explained Variance,0.3447 ± 0.0630
4,MAP@10,0.9843 ± 0.0038
5,NDCG@10,0.9882 ± 0.0028
6,Precision@10,0.1454 ± 0.0026
7,Recall@10,0.9971 ± 0.0020



=== Average Metrics (Mean ± Std) ===
RMSE: 0.8151 ± 0.0087
MAE: 0.7095 ± 0.0113
R-squared: -0.8907 ± 0.2790
Explained Variance: 0.3447 ± 0.0630
MAP@10: 0.9843 ± 0.0038
NDCG@10: 0.9882 ± 0.0028
Precision@10: 0.1454 ± 0.0026
Recall@10: 0.9971 ± 0.0020


<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>